In [ ]:
import numpy as np
import os
import plotly.graph_objects as go
from PIL import Image

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories

%load_ext autoreload
%autoreload 2

In [ ]:
kitti_seq = '0034'

if kitti_seq == '0027':
    start_idx = 1000
elif kitti_seq == '0028':
    start_idx = 1550
else:
    start_idx = 0

# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

In [ ]:
TRAJLEN = 2000
gps_spoofing_biases = np.zeros(TRAJLEN)  
attack_start_idx = 1000
gps_spoofing_biases[attack_start_idx:] = np.linspace(0, 1, TRAJLEN-attack_start_idx)  # Ramping attack

spoof_pos_1 = gt_enu[:TRAJLEN].copy()
spoof_pos_1[:,0] += 50 * gps_spoofing_biases
spoof_pos_2 = gt_enu[:TRAJLEN].copy()
spoof_pos_2[:,0] += 100 * gps_spoofing_biases
spoof_pos_3 = gt_enu[:TRAJLEN].copy()
spoof_pos_3[:,0] += 200 * gps_spoofing_biases

w = 3  # line width
sz = 12  # marker size

gt_traj = go.Scatter(x=gt_enu[:TRAJLEN,0], y=gt_enu[:TRAJLEN,1], hovertext=np.arange(TRAJLEN), name='Ground-truth', line=dict(width=w, color='black'))
spoof_traj_1 = go.Scatter(x=spoof_pos_1[:,0], y=spoof_pos_1[:,1], hovertext=np.arange(TRAJLEN), name='0.5 m/s ramp', line=dict(width=w, color='red', dash='dot'))
spoof_traj_2 = go.Scatter(x=spoof_pos_2[:,0], y=spoof_pos_2[:,1], hovertext=np.arange(TRAJLEN), name='1.0 m/s ramp', line=dict(width=w, color='red', dash='dashdot'))
spoof_traj_3 = go.Scatter(x=spoof_pos_3[:,0], y=spoof_pos_3[:,1], hovertext=np.arange(TRAJLEN), name='2.0 m/s ramp', line=dict(width=w, color='red', dash='dash'))

start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=sz, color='black'), showlegend=False)
spoof_start = go.Scatter(x=[gt_enu[attack_start_idx,0]], y=[gt_enu[attack_start_idx,1]], 
            name='Spoofing start', mode='markers', marker=dict(size=sz, color='red'), showlegend=True)
fig = go.Figure(data=[gt_traj, spoof_traj_1, spoof_traj_2, spoof_traj_3, start, spoof_start])
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')

fig.update_layout(font=dict(size=26))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

if kitti_seq == '0018':
    fig.update_xaxes(range=[-300, 280])
    fig.update_layout(legend=dict(x=0.05, y=0.98))

    satimg_path = os.path.join(os.getcwd(), '..', '..', 'images', 'satimgs', '0018_satimg.png')
    satimg = Image.open(satimg_path)
    fig.add_layout_image(
        dict(
            source=satimg,
            xref="x",
            yref="y",
            x=-300,
            y=406,
            sizex=580,
            sizey=595,
            sizing="stretch",
            opacity=0.75,
            layer="below")
    )
elif kitti_seq == '0027':
    fig.update_xaxes(range=[-100, 650])
    fig.update_layout(legend=dict(x=0.72, y=0.05))

    satimg_path = os.path.join(os.getcwd(), '..', '..', 'images', 'satimgs', '0027_satimg.png')
    satimg = Image.open(satimg_path)
    fig.add_layout_image(
        dict(
            source=satimg,
            xref="x",
            yref="y",
            x=-300,
            y=406,
            sizex=580,
            sizey=595,
            sizing="stretch",
            opacity=0.75,
            layer="below")
    )
elif kitti_seq == '0028':
    fig.update_xaxes(range=[-20, 500])
    fig.update_layout(legend=dict(x=0.68, y=0.98))

    
elif kitti_seq == '0034':
    fig.update_xaxes(range=[-50, 950])
    fig.update_layout(legend=dict(x=0.05, y=0.98))




fig.update_layout(margin=go.layout.Margin(
    l=40, #left margin
    r=40, #right margin
    b=40, #bottom margin
    t=40  #top margin
))
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

In [ ]:
fig.layout.xaxis.range

In [ ]:
fig.layout.yaxis.range

In [ ]:
from PIL import Image

satimg_path = os.path.join(os.getcwd(), '..', '..', 'images', '0018_satimg.png')
satimg = Image.open(satimg_path)

Using gnss_lib_py

In [ ]:
from chimera_fgo.util.io import read_gt

from gnss_lib_py.parsers.navdata import NavData
from gnss_lib_py.utils.visualizations import plot_map

In [ ]:
kitti_seq = '0034'

if kitti_seq == '0027':
    start_idx = 1000
elif kitti_seq == '0028':
    start_idx = 1550
else:
    start_idx = 0

# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

In [ ]:
gt_data = read_gt(gtpath)
gt_data = gt_data[start_idx:]

# Get ground truth positions
lla = gt_data[:,:3]

# Convert to ENU
ref_lla = lla[0]

In [ ]:
TRAJLEN = 2000
gps_spoofing_biases = np.zeros(TRAJLEN)  
attack_start_idx = 1000
gps_spoofing_biases[attack_start_idx:] = np.linspace(0, 1, TRAJLEN-attack_start_idx)  # Ramping attack

spoof_pos_1 = gt_enu[:TRAJLEN].copy()
spoof_pos_1[:,0] += 50 * gps_spoofing_biases
spoof_pos_2 = gt_enu[:TRAJLEN].copy()
spoof_pos_2[:,0] += 100 * gps_spoofing_biases
spoof_pos_3 = gt_enu[:TRAJLEN].copy()
spoof_pos_3[:,0] += 200 * gps_spoofing_biases

In [ ]:
from pymap3d.enu import enu2geodetic

# convert spoof_pos_1 from enu to lla
spoof_pos_1_lla = np.zeros((TRAJLEN, 3))
for i in range(TRAJLEN):
    spoof_pos_1_lla[i,:] = enu2geodetic(spoof_pos_1[i,0], spoof_pos_1[i,1], spoof_pos_1[i,2], ref_lla[0], ref_lla[1], ref_lla[2])

# convert spoof_pos_2 from enu to lla
spoof_pos_2_lla = np.zeros((TRAJLEN, 3))
for i in range(TRAJLEN):
    spoof_pos_2_lla[i,:] = enu2geodetic(spoof_pos_2[i,0], spoof_pos_2[i,1], spoof_pos_2[i,2], ref_lla[0], ref_lla[1], ref_lla[2])

# convert spoof_pos_3 from enu to lla
spoof_pos_3_lla = np.zeros((TRAJLEN, 3))
for i in range(TRAJLEN):
    spoof_pos_3_lla[i,:] = enu2geodetic(spoof_pos_3[i,0], spoof_pos_3[i,1], spoof_pos_3[i,2], ref_lla[0], ref_lla[1], ref_lla[2])

In [ ]:
data_gt = NavData()
data_gt["lat_gt_deg"] = lla[:TRAJLEN,0]
data_gt["lon_gt_deg"] = lla[:TRAJLEN,1]

data_sp1 = NavData()
data_sp1["lat_sp1_deg"] = spoof_pos_1_lla[::3,0]
data_sp1["lon_sp1_deg"] = spoof_pos_1_lla[::3,1]

data_sp2 = NavData()
data_sp2["lat_sp2_deg"] = spoof_pos_2_lla[::3,0]
data_sp2["lon_sp2_deg"] = spoof_pos_2_lla[::3,1]

data_sp3 = NavData()
data_sp3["lat_sp3_deg"] = spoof_pos_3_lla[::3,0]
data_sp3["lon_sp3_deg"] = spoof_pos_3_lla[::3,1]

fig = plot_map(data_sp1, data_sp2, data_sp3, data_gt)
fig.show()

In [ ]:
marker_size = 7

# sp1
fig.data[0].marker.color = '#f54278'
fig.data[0].marker.size = marker_size
#fig.data[0].marker.symbol = 'diamond'
# sp2
fig.data[1].marker.color = 'red'
fig.data[1].marker.size = marker_size
#fig.data[1].marker.symbol = 'square'
# sp3
fig.data[2].marker.color = '#f58742'
fig.data[2].marker.size = marker_size
#fig.data[2].marker.symbol = 'circle'
# gt
fig.data[3].marker.color = 'black'
fig.data[3].marker.size = marker_size

# Increase fig size
fig.update_layout(
    autosize=False,
    width=1600,
    height=900)

fig.show()